In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# imports

from google.colab import files
from os import listdir
from os.path import isfile, join
import re
from collections import Counter
from PIL import Image
import numpy as np
from keras.models import Model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
#summary e plot_model
#from keras.utils import plot_model, summary
from matplotlib.image import imread
from random import shuffle
from math import ceil

In [4]:
# variaveis

img_to_load = 300 # imagens a serem carregadas por vez (carregando as 10000 de uma vez sobrecarrega o sistema)
crop_size = 150 # altura e largura da imagem a ser enviada para o modelo
train = 0.9 # porcentagem de dados a ser usado para treino

In [5]:
#função para pegar só o centro da imagem

def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [ ]:
#carregar nome de todas as imagens

ER_path = '/gdrive/My Drive/IDAO/train/ER/'
ER_files = [f'/gdrive/My Drive/IDAO/train/ER/{f}' for f in listdir(ER_path) if isfile(join(ER_path, f))]

NR_path = '/gdrive/My Drive/IDAO/train/NR/'
NR_files = [f'/gdrive/My Drive/IDAO/train/NR/{f}' for f in listdir(NR_path) if isfile(join(NR_path, f))]

all_files = ER_files + NR_files
shuffle(all_files)

In [ ]:
# criar um dicionário para armazenar todos os dados separadamente
# isso é necessário para garantir que 70% de cada uma das classes serão usadas para treino

images = {'NR_1_keV': [], 'NR_6_keV': [], 'NR_20_keV': [], 'ER_3_keV': [], 'ER_10_keV': [], 'ER_30_keV': []}

In [ ]:
# coloca todas as strings das imagens no dicionário

for particle_type in images.keys():
  images[particle_type] = [img for img in all_files if particle_type in img]

for x in images.keys():
    images[x] = images[x]
    print(f"{x}: {len(images[x])}")

NR_1_keV: 2177
NR_6_keV: 2255
NR_20_keV: 2208
ER_3_keV: 2243
ER_10_keV: 2272
ER_30_keV: 2237


In [ ]:
# carrega 70% dos dados no X_train e 30% no X_test

X_train = []
X_test = []

for particle_type in images.keys():
    X_train = X_train + images[particle_type][:int(train*len(images[particle_type]))]
    X_test = X_test + images[particle_type][int(train*len(images[particle_type])):]
shuffle(X_train)
shuffle(X_test)

In [ ]:
# cria o Y_train e Y_test com base no X_train e X_test criado anteriormente

Y_train = [int(re.findall(r"\d+(?=_keV)", en)[0])/30.0 for en in X_train]
Y_test = [int(re.findall(r"\d+(?=_keV)", en)[0])/30.0 for en in X_test]

# teste sem dividir por 30
Y_train = [int(re.findall(r"\d+(?=_keV)", en)[0]) for en in X_train]
Y_test = [int(re.findall(r"\d+(?=_keV)", en)[0]) for en in X_test]

print(X_train[:10])
print(Y_train[:10])
print(X_test[:10])
print(Y_test[:10])
print(len(X_test))
print(len(Y_test))

['/gdrive/My Drive/IDAO/train/ER/0.2464424967765808__CYGNO_60_40_ER_3_keV_930V_30cm_IDAO_iso_crop_hist_pic_run2_ev167;1.png', '/gdrive/My Drive/IDAO/train/ER/-1.283082365989685__CYGNO_60_40_ER_3_keV_930V_30cm_IDAO_iso_crop_hist_pic_run5_ev221;1.png', '/gdrive/My Drive/IDAO/train/ER/-0.7183917164802551__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run5_ev255;1.png', '/gdrive/My Drive/IDAO/train/NR/0.07020000368356705__CYGNO_60_40_He_NR_6_keV_930V_30cm_SRIM_IDAO_iso_crop_hist_pic_run2_ev252;1.png', '/gdrive/My Drive/IDAO/train/ER/1.2236688137054443__CYGNO_60_40_ER_3_keV_930V_30cm_IDAO_iso_crop_hist_pic_run2_ev583;1.png', '/gdrive/My Drive/IDAO/train/NR/2.1810998916625977__CYGNO_60_40_He_NR_6_keV_930V_30cm_SRIM_IDAO_iso_crop_hist_pic_run2_ev980;1.png', '/gdrive/My Drive/IDAO/train/NR/0.2946000099182129__CYGNO_60_40_He_NR_20_keV_930V_30cm_SRIM_IDAO_iso_crop_hist_pic_run5_ev731;1.png', '/gdrive/My Drive/IDAO/train/ER/-1.3905295133590698__CYGNO_60_40_ER_3_keV_930V_30cm_IDAO_iso_cro

In [ ]:
# first model test

test_model = Sequential()

test_model.add(Conv2D(32, (3, 3), input_shape=(crop_size, crop_size, 1)))
test_model.add(Activation('relu'))
test_model.add(MaxPooling2D(pool_size=(2, 2)))

test_model.add(Conv2D(32, (3, 3)))
test_model.add(Activation('relu'))
test_model.add(MaxPooling2D(pool_size=(2, 2)))

test_model.add(Conv2D(64, (3, 3)))
test_model.add(Activation('relu'))
test_model.add(MaxPooling2D(pool_size=(2, 2)))

test_model.add(Flatten())

test_model.add(Dense(512, activation='relu'))
#test_model.add(Dropout(0.1))

test_model.add(Dense(256, activation='relu'))
test_model.add(Dense(128, activation='relu'))

test_model.add(Dense(64))
test_model.add(Activation('relu'))
#test_model.add(Dropout(0.5)) # treinar dpois com o dropout
test_model.add(Dense(1))
test_model.add(Activation('linear'))

test_model.compile(loss='mean_absolute_error',
              optimizer='adam', #rmsprop
              metrics=['mean_squared_error']) #MSE

In [ ]:
# second model test

second_model = Sequential()

second_model.add(Conv2D(32, (3, 3), input_shape=(crop_size, crop_size, 1)))
second_model.add(Activation('relu'))
second_model.add(MaxPooling2D(pool_size=(2, 2)))

second_model.add(Conv2D(64, (3, 3)))
second_model.add(Activation('relu'))
second_model.add(MaxPooling2D(pool_size=(2, 2)))

second_model.add(Conv2D(128, (3, 3)))
second_model.add(Activation('relu'))
second_model.add(MaxPooling2D(pool_size=(2, 2)))

second_model.add(Flatten())

#second_model.add(Dropout(0.1))

second_model.add(Dense(64))
second_model.add(Activation('relu'))
#second_model.add(Dropout(0.5)) # treinar dpois com o dropout
second_model.add(Dense(1))
second_model.add(Activation('linear'))

second_model.compile(loss='mean_absolute_error',
              optimizer='rmsprop', #rmsprop
              metrics=['accuracy'])

In [ ]:
# third model test

third_model = Sequential()

third_model.add(Conv2D(32, (3, 3), input_shape=(crop_size, crop_size, 1)))
third_model.add(Activation('relu'))
third_model.add(MaxPooling2D(pool_size=(2, 2)))

third_model.add(Conv2D(64, (3, 3)))
third_model.add(Activation('relu'))
third_model.add(MaxPooling2D(pool_size=(2, 2)))

third_model.add(Conv2D(64, (3, 3)))
third_model.add(Activation('relu'))
third_model.add(MaxPooling2D(pool_size=(2, 2)))

third_model.add(Conv2D(128, (3, 3)))
third_model.add(Activation('relu'))
third_model.add(MaxPooling2D(pool_size=(2, 2)))

third_model.add(Flatten())

#third_model.add(Dropout(0.1))

test_model.add(Dense(128, activation='relu'))
third_model.add(Dense(64))
third_model.add(Activation('relu'))
#third_model.add(Dropout(0.5)) # treinar dpois com o dropout
third_model.add(Dense(1))
third_model.add(Activation('linear'))

third_model.compile(loss='mean_absolute_error',
              optimizer='rmsprop', #rmsprop
              metrics=['accuracy'])

In [ ]:
# fourth model test

fourth_model = Sequential()

fourth_model.add(Conv2D(32, (3, 3), input_shape=(crop_size, crop_size, 1)))
fourth_model.add(Activation('relu'))

fourth_model.add(Conv2D(64, (3, 3)))
fourth_model.add(Activation('relu'))
fourth_model.add(MaxPooling2D(pool_size=(2, 2)))

fourth_model.add(Conv2D(64, (3, 3)))
fourth_model.add(Activation('relu'))
fourth_model.add(MaxPooling2D(pool_size=(2, 2)))

fourth_model.add(Flatten())

#fourth_model.add(Dropout(0.1))

test_model.add(Dense(128, activation='relu'))
fourth_model.add(Dense(64))
fourth_model.add(Activation('relu'))
#fourth_model.add(Dropout(0.5)) # treinar dpois com o dropout
fourth_model.add(Dense(1))
fourth_model.add(Activation('linear'))

fourth_model.compile(loss='mean_absolute_error',
              optimizer='adam', #rmsprop
              metrics=['accuracy'])

In [ ]:
# treina o modelo usando batches de 300 imagens por vez

for i in range(ceil(len(X_train)/img_to_load)):
#for i in range(ceil(1500/img_to_load)):
    begin = i*img_to_load
    limit = min(i*img_to_load + img_to_load, len(X_train))
    if begin == limit:
        break

    X_train_img = [crop_center(imread(file), crop_size, crop_size) for file in X_train[begin:limit]]
    Y_train_img = Y_train[begin:limit]
    print(f'loaded {begin} to {limit}')

    X_train_img = np.stack(X_train_img, axis=0)
    X_train_img = X_train_img.reshape((X_train_img.shape + (1,)))
    Y_train_img = np.stack(Y_train_img, axis=0).reshape((len(Y_train_img), 1))
    #print(X_train_img.shape)
    test_model.fit(X_train_img, Y_train_img, epochs=10, batch_size=50)

loaded 0 to 300
Epoch 1/10
6/6 [==============================] - 1s 34ms/step - loss: 11.4004 - accuracy: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 0s 27ms/step - loss: 10.2936 - accuracy: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 0s 24ms/step - loss: 10.3363 - accuracy: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 0s 25ms/step - loss: 9.9718 - accuracy: 0.0000e+00
Epoch 5/10
6/6 [==============================] - 0s 24ms/step - loss: 10.0440 - accuracy: 0.0000e+00
Epoch 6/10
6/6 [==============================] - 0s 25ms/step - loss: 10.2455 - accuracy: 0.0000e+00
Epoch 7/10
6/6 [==============================] - 0s 24ms/step - loss: 9.6392 - accuracy: 0.0000e+00
Epoch 8/10
6/6 [==============================] - 0s 24ms/step - loss: 9.8368 - accuracy: 0.0000e+00
Epoch 9/10
6/6 [==============================] - 0s 26ms/step - loss: 9.7282 - accuracy: 0.0000e+00
Epoch 10/10
6/6 [==============================] - 0s 25ms/step - loss

KeyboardInterrupt: ignored

In [ ]:
# treina o modelo usando batches de 300 imagens por vez

for i in range(ceil(len(X_train)/img_to_load)):
#for i in range(ceil(1500/img_to_load)):
    begin = i*img_to_load
    limit = min(i*img_to_load + img_to_load, len(X_train))
    if begin == limit:
        break

    X_train_img = [crop_center(imread(file), crop_size, crop_size) for file in X_train[begin:limit]]
    Y_train_img = Y_train[begin:limit]
    print(f'loaded {begin} to {limit}')

    X_train_img = np.stack(X_train_img, axis=0)
    X_train_img = X_train_img.reshape((X_train_img.shape + (1,)))
    Y_train_img = np.stack(Y_train_img, axis=0).reshape((len(Y_train_img), 1))
    #print(X_train_img.shape)
    second_model.fit(X_train_img, Y_train_img, epochs=10, batch_size=50)
    third_model.fit(X_train_img, Y_train_img, epochs=10, batch_size=50)
    fourth_model.fit(X_train_img, Y_train_img, epochs=10, batch_size=50)

loaded 0 to 300
Epoch 1/10
6/6 [==============================] - 1s 40ms/step - loss: 9.5211 - accuracy: 0.1133
Epoch 2/10
6/6 [==============================] - 0s 33ms/step - loss: 8.3650 - accuracy: 0.1500
Epoch 3/10
6/6 [==============================] - 0s 32ms/step - loss: 7.4757 - accuracy: 0.1500
Epoch 4/10
6/6 [==============================] - 0s 32ms/step - loss: 7.4319 - accuracy: 0.1500
Epoch 5/10
6/6 [==============================] - 0s 32ms/step - loss: 7.3625 - accuracy: 0.1500
Epoch 6/10
6/6 [==============================] - 0s 33ms/step - loss: 6.9896 - accuracy: 0.1500
Epoch 7/10
6/6 [==============================] - 0s 33ms/step - loss: 6.6902 - accuracy: 0.1500
Epoch 8/10
6/6 [==============================] - 0s 32ms/step - loss: 6.4571 - accuracy: 0.1500
Epoch 9/10
6/6 [==============================] - 0s 32ms/step - loss: 5.5404 - accuracy: 0.1500
Epoch 10/10
6/6 [==============================] - 0s 31ms/step - loss: 5.5065 - accuracy: 0.1500
Epoch 1/10
6/

In [ ]:
# treina o modelo usando batches de 300 imagens por vez

for i in range(ceil(len(X_train)/img_to_load)):
#for i in range(ceil(1500/img_to_load)):
    begin = i*img_to_load
    limit = min(i*img_to_load + img_to_load, len(X_train))
    if begin == limit:
        break

    X_train_img = [crop_center(imread(file), crop_size, crop_size) for file in X_train[begin:limit]]
    Y_train_img = Y_train[begin:limit]
    print(f'loaded {begin} to {limit}')

    X_train_img = np.stack(X_train_img, axis=0)
    X_train_img = X_train_img.reshape((X_train_img.shape + (1,)))
    Y_train_img = np.stack(Y_train_img, axis=0).reshape((len(Y_train_img), 1))
    #print(X_train_img.shape)
    third_model.fit(X_train_img, Y_train_img, epochs=10, batch_size=50)

In [ ]:
# gera um arquivo com os pesos do modelo

#test_model.save('saved_test_model')

INFO:tensorflow:Assets written to: saved_test_model/assets


In [ ]:
# usar se estiver usando google colab, isso gera um zip e faz o download do modelo

#!zip -r saved_test_model.zip saved_test_model
#files.download('saved_test_model.zip') 

  adding: saved_test_model/ (stored 0%)
  adding: saved_test_model/variables/ (stored 0%)
  adding: saved_test_model/variables/variables.data-00000-of-00001 (deflated 47%)
  adding: saved_test_model/variables/variables.index (deflated 69%)
  adding: saved_test_model/assets/ (stored 0%)
  adding: saved_test_model/saved_model.pb (deflated 90%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# usar se estiver usando google colab, isso descompacta um arquivo no diretório do notebook

#!unzip saved_test_model.zip

In [ ]:
# carrega um modelo salvo anteriormente

test = load_model('saved_test_model')

In [ ]:
# carrega as 1000 primeiras imagens usadas para testar

X_test_img = [crop_center(imread(file), crop_size, crop_size) for file in X_test[:1000]]
print(f'loaded test images')
Y_test_img = Y_test[:1000]
X_test_img = np.stack(X_test_img, axis=0)
X_test_img = X_test_img.reshape((X_test_img.shape + (1,)))
Y_test_img = np.stack(Y_test_img, axis=0).reshape((len(Y_test_img), 1))

loaded test images


In [ ]:
# valida o modelo
#test = modelo carregado
preds = test_model.evaluate(X_test_img, Y_test_img, batch_size=32, verbose=1, sample_weight=None)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

32/32 [==============================] - 1s 10ms/step - loss: 0.0070 - accuracy: 0.1660

Loss = 0.0070226178504526615
Test Accuracy = 0.16599999368190765


In [ ]:
# valida o modelo
#test = modelo carregado
preds = second_model.evaluate(X_test_img, Y_test_img, batch_size=32, verbose=1, sample_weight=None)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

32/32 [==============================] - 0s 10ms/step - loss: 11.8352 - accuracy: 0.1660

Loss = 11.835211753845215
Test Accuracy = 0.16599999368190765


In [ ]:
# valida o modelo
#test = modelo carregado
preds = third_model.evaluate(X_test_img, Y_test_img, batch_size=32, verbose=1, sample_weight=None)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

32/32 [==============================] - 0s 8ms/step - loss: 12.0839 - accuracy: 0.1660

Loss = 12.08386516571045
Test Accuracy = 0.16599999368190765


In [ ]:
# valida o modelo
#test = modelo carregado
preds = fourth_model.evaluate(X_test_img, Y_test_img, batch_size=32, verbose=1, sample_weight=None)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

32/32 [==============================] - 0s 13ms/step - loss: 7.3074 - accuracy: 0.1660

Loss = 7.307413101196289
Test Accuracy = 0.16599999368190765
